<a href="https://colab.research.google.com/github/milad-s5/DL_HW2/blob/Q1/DL_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load dataset and import libs